In [1]:
import os
from tqdm import tqdm
path = 'D:/ExportYolov5_20230330/datasets/train'
annotations = []
imgs = []
for root,dirs,files in os.walk(path):
    for file in files:
        if '.txt' in file :
            annotations.append(root+'/'+file)
        elif '.jpg' in file:
            imgs.append(root+'/'+file)
print(f'annotation file : {len(annotations)}')
print(f'image file : {len(imgs)}')

annotation file : 36421
image file : 36421


replace \t -> '  

In [39]:
for file in tqdm(annotations):
    new_text_content = ''

    with open(file,'r',encoding='UTF-8') as f :
        lines = f.readlines()
        for idx, line in enumerate(lines):
            new_string = line.strip().replace("\t"," ")
            if new_string:
                new_text_content += new_string + '\n'
            else :
                new_text_content += '\n'
        
    with open(file,'w',encoding='UTF-8') as f:
        f.write(new_text_content)

100%|██████████| 36421/36421 [00:14<00:00, 2541.58it/s]


## 전처리 
- max count : 1000
- basename이 같은 image 파일도 디렉토리 이동

In [4]:
from tqdm import tqdm
import shutil
import os

max_annotations = 1001
class_counts = {}
for file in tqdm(annotations):
    with open(file, 'r', encoding='UTF-8') as f:
        linelist = ''
        lines = f.readlines()

        for line in lines:
            class_id = line.split(' ')[0]
            if class_id not in class_counts:
                class_counts[class_id] = 0
            else :     
                class_counts[class_id] += 1
                if class_counts[class_id] < max_annotations:
                    linelist += line
                else : pass

        if len(linelist) > 0:
            with open(os.path.join('D:/ExportYolov5_20230330/datasets/valid/labels/', os.path.basename(file)), 'a+', encoding='UTF-8') as f:
                f.write(linelist)
            shutil.move(os.path.join('D:/ExportYolov5_20230330/datasets/train/images/', os.path.basename(file)[:-4]+'.jpg'),os.path.join('D:/ExportYolov5_20230330/datasets/valid/images/', os.path.basename(file)[:-4]+'.jpg'))
        else : pass


100%|██████████| 36421/36421 [01:46<00:00, 341.74it/s]


## 전처리 완료 데이터 점검

In [5]:
import os

path = 'D:/ExportYolov5_20230330/datasets/valid'
annotations = []
imgs = []
for root,dirs,files in os.walk(path):
    for file in files:
        if '.txt' in file :
            annotations.append(root+'/'+file)
        elif '.jpg' in file:
            imgs.append(root+'/'+file)
print(f'annotation file : {len(annotations)}')
print(f'image file : {len(imgs)}')

annotation file : 24697
image file : 24697


In [6]:
import os, csv
import pandas as pd
from collections import defaultdict
class_names =  ['WS_XR_CrossShape', 'WS_XR_Tshape', 'WS_XR_Yshape', 'WS_XR_RTshape', 'WS_XR_LTshape', 'WS_ThroughST', 'WS_Merge_R', 'WS_Merge_L', 'WS_Roundabout', 'WS_CrossRailroad', 'WS_R_Curve', 'WS_L_Curve', 'WS_RL_Curve', 'WS_LL_Curve', 'WS_2way', 'WS_Ascent_Road', 'WS_Descent_Road', 'WS_Narrow_Road', 'WS_Vanish_RightRoad', 'WS_Vanish_LeftRoad', 'WS_Pass_R', 'WS_Pass_Both', 'WS_MedianStrip_Start', 'WS_MedianStrip_End', 'WS_Flag', 'WS_Slippery', 'WS_Riverside', 'WS_UnevenSurface', 'WS_SpeedBump', 'WS_Rockslide', 'WS_Crosswalk', 'WS_Children', 'WS_Bicycle', 'WS_RC', 'WS_Airplane', 'WS_Sidewind', 'WS_Tunnel', 'WS_Bridge', 'WS_Wild_Animal', 'WS_Danger', 'WS_CCS', 'RS_E_TEMPPause', 'RS_E_Square', 'RS_E_Triangle', 'RS_E_TrafficCone', 'RS_E_Drum', 'RS_E_PEFence', 'RS_E_SignalVehicle', 'RS_E_ForkCrane', 'RS_E_PEFence2', 'RS_E_SCFence', 'RS_E_ETHeavy', 'I_Py_PEDmall', 'I_Py_Crosswalk', 'I_Py_P_Older', 'I_Py_P_Children', 'I_Py_P_DisablePerson', 'I_Py_BicycleCrossing', 'I_C_Road_Car', 'I_C_Road_Bicycle', 'I_C_Road_Bicycle_PED', 'I_C_Roundabout', 'I_C_STR', 'I_C_STR_RT', 'I_C_RT', 'I_C_LT', 'I_C_STR_LT', 'I_C_LT_Uturn', 'I_C_LT_RT', 'I_C_Uturn', 'I_C_Bothpass', 'I_C_Rightpass', 'I_C_Leftpass', 'I_C_Diversion', 'I_C_Bicycle_PED', 'I_C_Car_Bicycle', 'I_S_TrafficByDirection', 'I_S_Road_Bicycle', 'I_S_PL', 'I_S_CyclePL', 'I_S_Oneway', 'I_S_LT_Caution', 'I_S_Road_Bus', 'I_S_Road_HOV', 'I_S_Pass_first', 'TC_H_3color', 'TC_V_3color', 'TC_H_4color', 'TC_V_4color', 'TC_Y_Flasher', 'TC_Red_flasher', 'RS_T_Slow', 'RS_T_Yield', 'RS_C_PROH_Pass', 'RS_C_PROH_CarPass', 'RS_C_PROH_TruckPass', 'RS_C_PROH_OmnibusPass', 'RS_C_PROH_TWMV', 'RS_C_PROH_WMV', 'RS_C_PROH_TAH', 'RS_C_PROH_Bicycle', 'RS_C_PROH_Entry', 'RS_C_PROH_STR', 'RS_C_PROH_RT', 'RS_C_PROH_LT', 'RS_C_PROH_Uturn', 'RS_C_PROH_Overtaking', 'RS_C_PROH_TEMPStop', 'RS_C_PROH_STOP', 'RS_C_Limit_Weight', 'RS_C_Limit_Height', 'RS_C_Limit_Breadth', 'RS_C_DistanceWithCar', 'RS_C_Limit_MaxSpeed', 'RS_C_Limit_MinSpeed', 'RS_C_PROH_Walk', 'RS_C_PROH_LoadHazard']
class_counts = defaultdict(int)


for file in tqdm(annotations) :
    with open(file, 'r') as f:
        for line in f.readlines():
            class_id = int(line.split()[0])
            class_name = class_names[class_id]
            class_counts[class_name] += 1
for class_id, class_name in enumerate(class_names):
    count = class_counts[class_name]
    print(f"{class_name}, {class_id}, {count}")

100%|██████████| 24697/24697 [01:01<00:00, 400.57it/s]

WS_XR_CrossShape, 0, 0
WS_XR_Tshape, 1, 19
WS_XR_Yshape, 2, 2
WS_XR_RTshape, 3, 144
WS_XR_LTshape, 4, 827
WS_ThroughST, 5, 3
WS_Merge_R, 6, 517
WS_Merge_L, 7, 24
WS_Roundabout, 8, 13
WS_CrossRailroad, 9, 63
WS_R_Curve, 10, 274
WS_L_Curve, 11, 422
WS_RL_Curve, 12, 39
WS_LL_Curve, 13, 9
WS_2way, 14, 0
WS_Ascent_Road, 15, 22
WS_Descent_Road, 16, 20
WS_Narrow_Road, 17, 5
WS_Vanish_RightRoad, 18, 406
WS_Vanish_LeftRoad, 19, 0
WS_Pass_R, 20, 15
WS_Pass_Both, 21, 20
WS_MedianStrip_Start, 22, 9
WS_MedianStrip_End, 23, 0
WS_Flag, 24, 119
WS_Slippery, 25, 117
WS_Riverside, 26, 66
WS_UnevenSurface, 27, 324
WS_SpeedBump, 28, 714
WS_Rockslide, 29, 20
WS_Crosswalk, 30, 913
WS_Children, 31, 24
WS_Bicycle, 32, 59
WS_RC, 33, 38
WS_Airplane, 34, 0
WS_Sidewind, 35, 2
WS_Tunnel, 36, 4
WS_Bridge, 37, 0
WS_Wild_Animal, 38, 20
WS_Danger, 39, 305
WS_CCS, 40, 2
RS_E_TEMPPause, 41, 254
RS_E_Square, 42, 668
RS_E_Triangle, 43, 0
RS_E_TrafficCone, 44, 1000
RS_E_Drum, 45, 164
RS_E_PEFence, 46, 1000
RS_E_SignalVehic

## class분포 csv파일로 저장

In [7]:
output_csv = 'class_counts.csv'
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['classname', 'class_id', 'count'])

    for class_id, class_name in enumerate(class_names):
        count = class_counts[class_name]
        csv_writer.writerow([class_name, class_id, count])

print(f"Class counts saved to {output_csv}.")

Class counts saved to class_counts.csv.


## 전처리 후 다른 폴더로 옮긴 txt파일 삭제
   -> image와 매핑이 안되는 txt 파일 삭제

In [8]:
import os

img_path = 'D:/ExportYolov5_20230330/datasets/train/images'
anno_path = 'D:/ExportYolov5_20230330/datasets/train/labels'

imgs = []
annotations = []

for root, dirs, files in os.walk(img_path):
    for file in files:
        if file.endswith('.jpg'):
            imgs.append(os.path.join(root, file))

for root, dirs, files in os.walk(anno_path):
    for file in files:
        if file.endswith('.txt'):
            annotations.append(os.path.join(root, file))

print(f'annotation files: {len(annotations)}')
print(f'image files: {len(imgs)}')

for anno in tqdm(annotations):
    corresponding_img = os.path.splitext(anno)[0].replace('labels', 'images') + '.jpg'
    if corresponding_img not in imgs:
        os.remove(anno)

annotation files: 36421
image files: 11724


100%|██████████| 36421/36421 [00:08<00:00, 4139.69it/s]
